In [1]:
# Autoreload extension for interactive development
%load_ext autoreload
%autoreload 2

# Standard library imports
import json
import os

# Local application/library specific imports
from pollo.utils.hit import HitMaker
import pollo.utils.io as io_utils

ModuleNotFoundError: No module named 'pollo'

### Create and save some HITs as JSON files

In [ ]:
class ImageHitMaker(HitMaker):
    """Image hit maker."""

In [ ]:
ENDPOINT = "https://mturk.ethanweber.me"
MOVIE_FOLDER = "https://mturk.ethanweber.me/static/data/media/nerfiller_videos_occluder"
HIT_FOLDER = "/home/ethanweber/anno/static/data/hits"
dataset_names = [
    "billiards",
    "dumptruck",
    "office",
    "backpack",
    "drawing",
    "norway",
    "boot",
    "bear",
    "cat",
    "turtle",
]
baseline_names = [
    "individual-lama-individual-inpaint-once", # LaMask
    "individual-sd-image-individual-inpaint-once", # SD Image Cond
    "none-individual-inpaint-du", # Inpaint + DU
]
mask_name = "none-nerfacto-nerfiller-visualize"
our_name = "none-grid-prior-du"
hit_maker = NerfillerHitMaker()

baseline_to_query_examples = {}
hits_per_baseline = 20

In [ ]:
# for baseline_name in baseline_names:
#     for i in range(hits_per_baseline):
#         query_examples = []
#         for dataset_name in dataset_names:
#             choices = [
#                 f"https://mturk.ethanweber.me/static/data/media/nerfiller_videos_occluder/{dataset_name}-{baseline_name}.mp4",
#                 f"https://mturk.ethanweber.me/static/data/media/nerfiller_videos_occluder/{dataset_name}-{our_name}.mp4",
#             ]
#             query_examples.append({
#                 "video_to_inpaint": f"https://mturk.ethanweber.me/static/data/media/nerfiller_videos_occluder/{dataset_name}-{mask_name}.mp4",
#                 "choices": choices,
#             })
#         for dataset_name in dataset_names:
#             choices_reversed = [
#                 f"https://mturk.ethanweber.me/static/data/media/nerfiller_videos_occluder/{dataset_name}-{our_name}.mp4",
#                 f"https://mturk.ethanweber.me/static/data/media/nerfiller_videos_occluder/{dataset_name}-{baseline_name}.mp4",
#             ]
#             query_examples.append({
#                 "video_to_inpaint": f"https://mturk.ethanweber.me/static/data/media/nerfiller_videos_occluder/{dataset_name}-{mask_name}.mp4",
#                 "choices": choices_reversed
#             })
#         hit = hit_maker.get_hit(
#             gt_query_examples=[],
#             query_examples=query_examples
#         )
#         config_name = f"hit-{baseline_name}-{i:06d}"
#         # print(config_name)
#         write_to_json(os.path.join(HIT_FOLDER, config_name + ".json"), hit)

### Launch some HITs and monitor progress

In [ ]:
from anno.requester import Requester

In [ ]:
use_sandbox = False
DATABASE_FILENAME = "mturk_database.pkl"
CREDS_FILENAME = "mturk_creds.json"

requester = Requester(use_sandbox=use_sandbox,
                      database_filename=DATABASE_FILENAME,
                      creds_filename=CREDS_FILENAME)
requester.show_account_balance()

In [ ]:
lifetime_hours = 1
per_hit_minutes = 15
cost_per_hit = '0.5'
days_until_autoapprove = 2
estimated_time_in_minutes = 10
task_attributes = {
    # Number of assignments per task.
    'MaxAssignments': 1,
    # How long the task will be available on MTurk
    'LifetimeInSeconds': 60 * 60 * lifetime_hours,
    # How long Workers have to complete each item
    'AssignmentDurationInSeconds': 60 * per_hit_minutes,
    # The reward you will offer Workers for each response
    'Reward': cost_per_hit,
    'Title': 'Choosing which video looks better. (~{} min)'.format(estimated_time_in_minutes),
    'Keywords': 'video, choice, computer vison, selection',
    'Description': 'Choose which of the two videos looks better.',
    'AutoApprovalDelayInSeconds': days_until_autoapprove * (60*60*24)
}
requester.set_task_attributes(task_attributes)

In [ ]:
# existing_external_urls = set(external_urls)

In [ ]:
# existing_external_urls

In [ ]:
external_urls = []
for filename in sorted(os.listdir(HIT_FOLDER)):
    external_url = os.path.join(ENDPOINT, "interface", filename.replace(".json", ""))
    external_urls.append(external_url)

In [ ]:
hit_ids = requester.submit_hits_from_external_urls(external_urls)

In [ ]:
requester.update_database_with_responses()

In [ ]:
responses = requester.get_responses_from_database()
len(responses)

In [ ]:
# minutes per HIT on average
requester.get_mean_time_from_resonses(responses) / 60

In [ ]:
requester.save_all_responses_to_files(responses)

In [ ]:
# process all the results

responses_path = "/home/ethanweber/nerfiller-user-study/static/data/responses"
responses_filenames = sorted(os.listdir(responses_path))

from collections import defaultdict
baseline_to_choice = defaultdict(list)
baseline_to_dataset_name = defaultdict(list)

for response_filename in responses_filenames:
    response = json.load(open(os.path.join(responses_path, response_filename)))
    num_examples = len(response["QUERY_EXAMPLES_RESPONSES"])
    for example_idx in range(num_examples):
        answer_idx = int(response["QUERY_EXAMPLES_RESPONSES"][example_idx])
        global_config_name = response["GLOBAL_CONFIG_NAME"]
        baseline_name = global_config_name[4:global_config_name.find("-0000")]
        choice = response["GLOBAL_CONFIG"]["QUERY_EXAMPLES"][example_idx]["choices"][answer_idx]
        dataset_idx_s = choice.find("nerfiller_videos_occluder") + len("nerfiller_videos_occluder/")
        dataset_idx_e = choice.find("-", dataset_idx_s)
        dataset_name = choice[dataset_idx_s:dataset_idx_e]
        chosen_method = choice[dataset_idx_e+1:-4]
        baseline_to_choice[baseline_name].append(chosen_method)
        baseline_to_dataset_name[baseline_name].append(dataset_name)
        # print(baseline_name, ":::", dataset_name, ":::", chosen_method)

In [ ]:
scene_dataset_names = [
    "billiards",
    # "dumptruck",
    "office",
    "backpack",
    "drawing",
    "norway",
    # "boot",
    # "bear",
    # "cat",
    # "turtle",
]
object_dataset_names = [
    # "billiards",
    "dumptruck",
    # "office",
    # "backpack",
    # "drawing",
    # "norway",
    "boot",
    "bear",
    "cat",
    "turtle",
]
dataset_filter = set(dataset_names)

num = 0
den = 0
baseline_name = "individual-lama-individual-inpaint-once"
for i in range(len(baseline_to_choice[baseline_name])):
    value = baseline_to_choice[baseline_name][i]
    dataset_name = baseline_to_dataset_name[baseline_name][i]
    if dataset_name not in dataset_filter:
        continue
    num += 1 if value == "none-grid-prior-du" else 0
    den += 1
print(baseline_name, num/den)

num = 0
den = 0
baseline_name = "individual-sd-image-individual-inpaint-once"
for i in range(len(baseline_to_choice[baseline_name])):
    value = baseline_to_choice[baseline_name][i]
    dataset_name = baseline_to_dataset_name[baseline_name][i]
    if dataset_name not in dataset_filter:
        continue
    num += 1 if value == "none-grid-prior-du" else 0
    den += 1
print(baseline_name, num/den)

num = 0
den = 0
baseline_name = "none-individual-inpaint-du"
for i in range(len(baseline_to_choice[baseline_name])):
    value = baseline_to_choice[baseline_name][i]
    dataset_name = baseline_to_dataset_name[baseline_name][i]
    if dataset_name not in dataset_filter:
        continue
    num += 1 if value == "none-grid-prior-du" else 0
    den += 1
print(baseline_name, num/den)

In [ ]:
# worker_ids = []
# for hit_id in responses.keys():
#     status = requester.client.get_hit(HITId=hit_id)
#     listed_responses = requester.client.list_assignments_for_hit(HITId=hit_id)
#     worker_id = listed_responses["Assignments"][0]["WorkerId"]
#     print()
#     print("worker_id:", worker_id)
#     print()
#     worker_ids.append(worker_id)
#     assert len(responses[hit_id])